1. 일기 생성 후 감정 분류: KoBERT 모델로 일기의 감정을 분류.

2. 트로트 노래 데이터 크롤링 및 라벨링: 멜론에서 트로트 노래 제목과 가사를 크롤링하고, KoBERT 모델로 가사를 감정 라벨링.

3. E5 임베딩 활용: Microsoft의 E5 임베딩 모델을 사용하여 일기와 노래 가사의 임베딩 벡터를 생성.

4. 유사도 계산 및 추천: 일기의 임베딩과 감정이 동일한 노래 중 가장 유사한 5개의 노래를 추천.

In [1]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer
import re

class EmotionClassifier:
    def __init__(self, model_path="monologg/kobert", num_labels=7, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code = True) # trust_remote_code = 모델 다운로드에 대한 검증 절차가 생략
        self.model.to(self.device)

        # 감정매핑
        self.label_to_emotion = {
            0: "중립",
            1: "놀람",
            2: "분노",
            3: "슬픔",
            4: "행복",
            5: "혐오",
            6: "공포"
        }

    def load_model(self, model_file):
        """학습된 감정분류 모델 불러오기"""
        self.model.load_state_dict(torch.load(model_file))

    def preprocess_text(self, text):
        """텍스트 전처리"""
        return re.sub("[^0-9a-zA-Z가-힣\s+]", "", text)

    def predict_emotion(self, text):
        """감정 분류 및 예측"""
        # 텍스트 입력 / 토큰화 / 분류
        cleaned_text = self.preprocess_text(text)
        encoded_input = self.tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # 예측하기
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**encoded_input)
            predicted_label = outputs.logits.argmax(dim=1).item()
        
        # 분류된 감정 라벨
        predicted_emotion = self.label_to_emotion[predicted_label]
        
        return predicted_emotion


## Microft E5 임베딩 모델

In [2]:
from transformers import AutoTokenizer, AutoModel

class E5Embedder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-large")
        self.model = AutoModel.from_pretrained("intfloat/e5-large")

    def get_embedding(self, text):
        """텍스트를 E5 임베딩 벡터로 변환"""
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
        outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).detach()


## 유사도 계산 및 추천 시스템

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class SongRecommender:
    def __init__(self, songs):
        """
        songs: 크롤링된 트로트 노래 데이터 (리스트 형태)
               각 항목은 {"title": 제목, "lyrics": 가사, "emotion": 감정, "embedding": 임베딩} 형태로 구성
        """
        self.songs = songs

    def recommend_songs(self, diary_embedding, emotion, top_k=5):
        """
        감정이 동일한 노래 중에서 일기와 가장 유사한 노래 추천
        """
        # 동일한 감정의 노래 필터링
        filtered_songs = [song for song in self.songs if song["emotion"] == emotion]

        # 코사인 유사도 계산
        similarities = []
        for song in filtered_songs:
            similarity = cosine_similarity(diary_embedding.numpy(), song["embedding"].numpy())
            similarities.append((song["title"], similarity[0][0]))

        # 유사도 순으로 정렬하여 상위 K개 선택
        recommendations = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
        
        return recommendations


## 테스트

In [6]:
if __name__ == "__main__":
    # KoBERT 감정 분류 모델 초기화 및 로드
    emotion_model_path = "new_data_test.pth"
    emotion_classifier = EmotionClassifier()
    emotion_classifier.load_model(emotion_model_path)

    # E5 임베딩 모델 초기화
    embedder = E5Embedder()

    # 크롤링된 트로트 노래 데이터 (예시 데이터)
    songs_data = [
    {
        "title": "진정인가요",
        "lyrics": "미련없다 그 말이 진정인가요 냉정했던 그 마음이 진정인가요 바닷가를 거닐며 수놓았던 그 추억 잊을 수가 있을까요 돌아설 때 눈물짓던 당신이라면 영원토록 영원토록 죽어도 못 잊을 겁니다 잊어달라 그 말이 진정인가요 돌아서던 그 마음이 진정인가요 오솔길을 거닐며 주고받던 그 사랑 잊을 수가 있을까요 돌아설 때 울음 참던 당신이라면 잊으려도 잊으려도 죽어도 못 잊을 겁니다 죽어도 못 잊을 겁니다",
        "emotion": "슬픔",
        "embedding": embedder.get_embedding("미련없다 그 말이 진정인가요 냉정했던 그 마음이 진정인가요 바닷가를 거닐며 수놓았던 그 추억 잊을 수가 있을까요 돌아설 때 눈물짓던 당신이라면 영원토록 영원토록 죽어도 못 잊을 겁니다 잊어달라 그 말이 진정인가요 돌아서던 그 마음이 진정인가요 오솔길을 거닐며 주고받던 그 사랑 잊을 수가 있을까요 돌아설 때 울음 참던 당신이라면 잊으려도 잊으려도 죽어도 못 잊을 겁니다 죽어도 못 잊을 겁니다")
    },
    {
        "title": "빈잔",
        "lyrics": "마음은 비우라고 했는데 왜 이리 허전할까 빈잔에 술을 채워 마셔봐도 채워지지 않는 내 마음 슬픔은 또 한잔의 술로 달래고자 하는데 빈잔에 남은 내 마음은 더 허전해지네",
        "emotion": "중립",
        "embedding": embedder.get_embedding("마음은 비우라고 했는데 왜 이리 허전할까 빈잔에 술을 채워 마셔봐도 채워지지 않는 내 마음 슬픔은 또 한잔의 술로 달래고자 하는데 빈잔에 남은 내 마음은 더 허전해지네")
    },
    {
        "title": "막걸리 한 잔",
        "lyrics": "아버지 우리 아버지 산골에서 막걸리 한잔 아버지 우리 아버지 산골에서 막걸리 한잔 자식들 생각하며 고생만 하셨네 아버지 우리 아버지 산골에서 막걸리 한잔",
        "emotion": "놀람",
        "embedding": embedder.get_embedding("아버지 우리 아버지 산골에서 막걸리 한잔 아버지 우리 아버지 산골에서 막걸리 한잔 자식들 생각하며 고생만 하셨네 아버지 우리 아버지 산골에서 막걸리 한잔")
    },
    {
        "title": "미운 사랑",
        "lyrics": "사랑이란 게 참 미운 거야 너를 사랑하면서도 미워하니까 미운 사랑에 울고 웃으며 오늘도 너를 기다린다",
        "emotion": "분노",
        "embedding": embedder.get_embedding("사랑이란 게 참 미운 거야 너를 사랑하면서도 미워하니까 미운 사랑에 울고 웃으며 오늘도 너를 기다린다")
    },
    {
        "title": "사랑의 배터리",
        "lyrics": "당신 없인 못 살아 정말 나는 못 살아 당신은 나의 배터리 당신 없인 못 살아 정말 나는 못 살아 당신은 나의 햇살 같은 존재",
        "emotion": "행복",
        "embedding": embedder.get_embedding("당신 없인 못 살아 정말 나는 못 살아 당신은 나의 배터리 당신 없인 못 살아 정말 나는 못 살아 당신은 나의 햇살 같은 존재")
    },
    {
        "title": "애모",
        "lyrics": "그대 나를 위해 웃음을 보여도 허탈한 표정 감출 순 없어 힘없이 뒤돌아서는 그대의 모습을 흐린 눈으로 바라만 보네 나는 알고 있어요 우리의 사랑은 이것이 마지막이라는 것을",
        "emotion": "혐오",
        "embedding": embedder.get_embedding("그대 나를 위해 웃음을 보여도 허탈한 표정 감출 순 없어 힘없이 뒤돌아서는 그대의 모습을 흐린 눈으로 바라만 보네 나는 알고 있어요 우리의 사랑은 이것이 마지막이라는 것을")
    },
    {
        "title": "공포의 외인구단",
        "lyrics": "밤하늘에 별빛처럼 빛나는 우리의 꿈 공포의 외인구단 승리를 향해 달려간다 우리는 하나다 우리는 강하다 모두가 두려워하는 공포의 외인구단",
        "emotion": "공포",
        "embedding": embedder.get_embedding("밤하늘에 별빛처럼 빛나는 우리의 꿈 공포의 외인구단 승리를 향해 달려간다 우리는 하나다 우리는 강하다 모두가 두려워하는 공포의 외인구단")
    }
]


    # 추천 시스템 초기화
    recommender = SongRecommender(songs_data)

    # 사용자 입력 처리
    user_input = input("오늘의 일기를 입력하세요: ")
    
    # 감정 예측
    predicted_emotion = emotion_classifier.predict_emotion(user_input)

    # 사용자 입력 텍스트의 임베딩 생성
    diary_embedding = embedder.get_embedding(user_input)

    # 노래 추천 실행
    recommendations = recommender.recommend_songs(diary_embedding, predicted_emotion)

    # 결과 출력
    print("\n--- 추천 결과 ---")
    print(f"예측된 감정: {predicted_emotion}")
    print("추천 노래:")
    for title, similarity in recommendations:
        print(f"- {title} (유사도: {similarity:.4f})")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- 추천 결과 ---
예측된 감정: 공포
추천 노래:
- 공포의 외인구단 (유사도: 0.9161)
